<a href="https://colab.research.google.com/github/HilbertN/Redes_Neuronales/blob/main/RN8_MLFlowandDagsHub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
REPO_NAME= "hello-world"
REPO_OWNER= "HilbertN"
USER_NAME = "HilbertN"


In [15]:
!pip install mlflow --quiet

In [3]:
import mlflow
import os
from getpass import getpass

os.environ['MLFLOW_TRACKING_USERNAME'] = USER_NAME
os.environ['MLFLOW_TRACKING_PASSWORD'] = getpass('Enter your DAGsHub access token or password: ')

mlflow.set_tracking_uri(f'https://dagshub.com/{REPO_OWNER}/{REPO_NAME}.mlflow')

Enter your DAGsHub access token or password: ··········


In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input
from tensorflow.keras.optimizers import RMSprop, SGD, Adam
from tensorflow.keras import regularizers
from keras.callbacks import ModelCheckpoint, EarlyStopping



In [5]:
dataset=mnist.load_data()
(x_train, y_train), (x_test, y_test) = dataset

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255  # x_trainv = x_trainv/255
x_test /= 255

num_classes=10
y_trainc = keras.utils.to_categorical(y_train, num_classes)
y_testc = keras.utils.to_categorical(y_test, num_classes)

11490434/11490434 [==============================] - 1s 0us/step


In [6]:
model = Sequential()
model.add(Input(shape=(28,28))) #Flaten no tiene la opcion input_shape por lo tanto se tiene que agregar esta capa
model.add(Flatten()) #Otra forma de aplanar las imagenes
model.add(Dense(512, activation='sigmoid'))
model.add(Dense(512, activation='relu', kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4))) #Regularizacion L1L2
model.add(Dropout(0.5)) #Fraccion de enlaces a eliminar
model.add(Dense(200)) #Capa lineal , transformacion lineal sin funcion de activacion
model.add(Activation('tanh')) #Se puede agregar despues la funcion de activacion
model.add(Dense(400, activation='selu', kernel_regularizer=regularizers.L1(0.01) )) #Regularizacion L1
model.add(Dense(200, activation='elu', kernel_regularizer=regularizers.L2(l2=1e-4)) ) #Regularizacion L2
model.add(Dense(50,activation='exponential'))
model.add(Dense(10, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 512)               401920    
                                                                 
 dense_1 (Dense)             (None, 512)               262656    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_2 (Dense)             (None, 200)               102600    
                                                                 
 activation (Activation)     (None, 200)               0         
                                                                 
 dense_3 (Dense)             (None, 400)               8

In [7]:
mlflow.tensorflow.autolog()


In [8]:
# specify the path where you want to save the model
filepath = "best_model.hdf5"

# initialize the ModelCheckpoint callback
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

El callback "EarlyStopping" hace que MLFlow pueda registrar el mejor modelo antes de sobre ajustar o que deje de mejorar el entrenamiento.

En resumen, con esta configuración de EarlyStopping, el entrenamiento del modelo se detendrá si la pérdida en el conjunto de validación no mejora durante 10 épocas consecutivas, y se restaurarán los pesos del modelo a la mejor iteración para garantizar que se retenga el mejor rendimiento del modelo.

In [9]:
earlystop = EarlyStopping(monitor='val_loss',mode='min',restore_best_weights=True, patience=10,verbose=1)

In [11]:
mlflow.set_experiment(experiment_name="Experiment 1")

MlflowException: ignored

In [12]:
Pueden correr varias corridas en el mismo experimento con la opcion : nested='TRUE'

SyntaxError: ignored

In [13]:
mlflow.start_run(nested='TRUE')
model.compile(loss="categorical_crossentropy",optimizer=Adam(learning_rate=0.0001),metrics=['accuracy'])
model.fit(x_train, y_trainc,
                    batch_size=128,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_testc),
                    callbacks=[earlystop, checkpoint])
mlflow.end_run()

MlflowException: ignored

Pueden tambien registrar el modelo con el que concluye el experimento.

In [14]:
mlflow.keras.save_model(model, f'https://dagshub.com/{REPO_OWNER}/{REPO_NAME}.mlflow')

AttributeError: ignored

Pueden tambien usar el callback ModelCheckpoint, pero tienen que registrar con el siguiente comando el modelo:

In [ ]:
mlflow.log_artifact('best_model.hdf5', "models")